# Ungraded Lab: Building a Custom Dense Layer

<a target="_blank" href="https://colab.research.google.com/github/LuisAngelMendozaVelasco/TensorFlow-Advanced_Techniques_Specialization/blob/master/Custom_Models_Layers_and_Loss_Functions_with_TensorFlow/Week3/Labs/C1_W3_Lab_2_custom-dense-layer.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Run in Google Colab</a>

In this lab, we'll walk through how to create a custom layer that inherits the [Layer](https://keras.io/api/layers/base_layer/#layer-class) class. Unlike simple Lambda layers you did previously, the custom layer here will contain weights that can be updated during training.

## Imports

In [1]:
import tensorflow as tf
import numpy as np
from keras import layers, Sequential, Input

2024-08-22 01:15:07.841885: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-22 01:15:07.854519: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-22 01:15:07.857963: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-22 01:15:07.866876: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Custom Layer with weights

To make custom layer that is trainable, we need to define a class that inherits the [Layer](https://keras.io/api/layers/base_layer/#layer-class) base class from Keras. The Python syntax is shown below in the class declaration. This class requires three functions: `__init__()`, `build()` and `call()`. These ensure that our custom layer has a *state* and *computation* that can be accessed during training or inference.

In [2]:
# Inherit from this base class
class SimpleDense(layers.Layer):
    def __init__(self, units=32):
        '''Initializes the instance attributes'''

        super(SimpleDense, self).__init__()
        self.units = units

    def build(self, input_shape):
        '''Create the state of the layer (weights)'''

        # Initialize the weights
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name="kernel", 
                             initial_value=w_init(shape=(input_shape[-1], self.units), dtype='float32'),
                             trainable=True)

        # Initialize the biases
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name="bias",
                             initial_value=b_init(shape=(self.units,), dtype='float32'),
                             trainable=True)

    def call(self, inputs):
        '''Defines the computation from inputs to outputs'''
        
        return tf.matmul(inputs, self.w) + self.b

Now we can use our custom layer like below:

In [3]:
# Declare an instance of the class
my_dense = SimpleDense(units=1)

# Define an input and feed into the layer
x = tf.ones((1, 1))
y = my_dense(x)

# Parameters of the base Layer class like `variables` can be used
print(my_dense.w, my_dense.b)

<tf.Variable 'simple_dense/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[-0.06089277]], dtype=float32)> <tf.Variable 'simple_dense/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>


2024-08-22 01:15:09.667990: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1822 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5


Let's then try using it in simple network:

In [4]:
# Define the dataset
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

# Use the Sequential API to build a model with our custom layer
my_layer = SimpleDense(units=1)
model = Sequential([Input(shape=(1,)),
                    my_layer])

# Configure and train the model
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(xs, ys, epochs=500, verbose=0)

# Perform inference
print(model.predict(np.array([10.0])))

# See the updated state of the variables
print(my_layer.variables)

/home/luis-mendoza/miniconda3/envs/TensorFlow/lib/python3.11/site-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")
I0000 00:00:1724310909.850576  290003 service.cc:146] XLA service 0x7e6660015370 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1724310909.850604  290003 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2024-08-22 01:15:09.864292: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
I0000 00:00:1724310909.892285  290003 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
[[-0.02387703]]
[]
